In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from shutil import copyfile
import shutil
import matplotlib.pyplot as plt
import cv2

In [ ]:
print(len(os.listdir('/gdrive/MyDrive/AML_FashionDataset/topwear')))
print(len(os.listdir('/gdrive/MyDrive/AML_FashionDataset/bottomwear')))
print(len(os.listdir('/gdrive/MyDrive/AML_FashionDataset/footwear')))

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from PIL import Image
from skimage.transform import resize


#### Resizing images and converting to grayscale (Part 3 preprocessing)
<br>
def load_and_preprocess_images(*folder_path):<br>
    image_files = []<br>
    images = []<br>
    labels = []<br>
    for fp in folder_path:<br>
      image_files = sorted(os.listdir(fp))<br>
      for filename in image_files:<br>
        img = Image.open(os.path.join(fp, filename))<br>
        img = img.convert('L')<br>
        img = img.resize((224,224))<br>
        img_array = np.array(img)<br>
        images.append(img_array)<br>
        if 'topwear' in filename:<br>
          label = 1<br>
        if 'bottomwear' in filename:<br>
          label = 2<br>
        if 'footwear' in filename:<br>
          label = 3<br>
        labels.append(label)<br>
    return images, labels<br>
import os<br>
import numpy as np<br>
import tensorflow as tf<br>
from keras import layers, models<br>
from sklearn.model_selection import train_test_split<br>
from sklearn.preprocessing import LabelEncoder<br>
from sklearn.metrics import accuracy_score<br>
from scipy.spatial import procrustes<br>
import os<br>
import numpy as np<br>
import matplotlib.pyplot as plt<br>
from sklearn.decomposition import PCA<br>
from PIL import Image<br>
from tensorflow.keras.preprocessing.image import ImageDataGenerator<br>
from tensorflow.keras.models import Sequential<br>
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, ZeroPadding2D<br>
from tensorflow.keras.optimizers import Adam<br>
from tensorflow.keras import callbacks<br>
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler<br>
import time<br>
np.random.seed(42)<br>
tf.random.set_seed(42)<br>
images, labels = load_and_preprocess_images('/gdrive/MyDrive/AML_FashionDataset/topwear/',<br>
                                            '/gdrive/MyDrive/AML_FashionDataset/bottomwear/',<br>
                                            '/gdrive/MyDrive/AML_FashionDataset/footwear/')<br>
all_images = np.array(images)<br>
all_labels = np.array(labels)<br>
all_labels = np.array(pd.get_dummies(all_labels))<br>
 Part 2: Split the images into a training set, a validation set, and a test set.


In [ ]:
from sklearn.utils import shuffle

In [ ]:
X, y = shuffle(all_images, all_labels, random_state=42)  # shuffle to ensure that the images of each class occur one after the other in a random order rather than class 0 being followed by class 1 and then class 2
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42) # stratify to ensure that equal proportions of each class are present in each set
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, stratify=y_test_val, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

In [ ]:
from collections import Counter

In [ ]:
train_lab_list = []
for i in y_train:
  train_lab_list.append(list(i).index(1))
print(Counter(train_lab_list))

In [ ]:
val_lab_list = []
for i in y_val:
  val_lab_list.append(list(i).index(1))
print(Counter(val_lab_list))

In [ ]:
test_lab_list = []
for i in y_test:
  test_lab_list.append(list(i).index(1))
print(Counter(test_lab_list))


# Part 3: Build the input pipeline, including the appropriate preprocessing operations, and add data augmentation.
<br>
image_size = 224<br>
X_train = X_train.reshape(-1, image_size, image_size, 1)<br>
X_val = X_val.reshape(-1, image_size, image_size, 1)<br>
X_test = X_test.reshape(-1, image_size, image_size, 1)<br>
# y_train = np.array(pd.get_dummies(y_train))<br>
# y_val = np.array(pd.get_dummies(y_val))<br>
# y_test = np.array(pd.get_dummies(y_test))<br>
datagen_train = ImageDataGenerator(<br>
    rotation_range=180,<br>
    zoom_range=0.2,<br>
    width_shift_range=0.2,<br>
    height_shift_range=0.2,<br>
    horizontal_flip=True,<br>
    vertical_flip=True  # Allowing vertical flip as well<br>
)<br>
 Part 4: Fine-tune a pretrained model of your choice on this dataset (the one you created in part 3). Report classification accuracy and give a few examples of correct/incorrect classification (show a few images that were correctly/incorrectly classified).


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
image_paths = []
image_labels = []

In [ ]:
categories = ['topwear','bottomwear','footwear']
for category in categories:
    category_dir = os.path.join('/gdrive/MyDrive/AML_FashionDataset/', category)

    # Get a list of all image files in the category
    category_images = [os.path.join(category_dir, img) for img in os.listdir(category_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Add image paths to the list
    image_paths.extend(category_images)

    # Add labels (category) to the list
    image_labels.extend([category] * len(category_images))

Use train_test_split to split the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, image_labels, test_size=0.2, random_state=42)

Further split the training set into training and validation sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2 (20% for validation)

Display the number of samples in each set

In [ ]:
print("Number of training samples:", len(X_train))
print("Number of validation samples:", len(X_val))
print("Number of testing samples:", len(X_test))

Assuming you have a DataFrame df with 'image_path' and 'label' columns

In [ ]:
df_train = pd.DataFrame({'image_path': X_train, 'label': y_train})
df_val = pd.DataFrame({'image_path': X_val, 'label': y_val})
df_test = pd.DataFrame({'image_path': X_test, 'label': y_test})

Define image size and batch size

In [ ]:
img_size = (224, 224)
batch_size = 32

Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

Create data generators

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

In [ ]:
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

Create the test generator

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Set to False to maintain the order
)

Load pre-trained models

rying with ResNet50 & then InceptionV3

In [ ]:
base_models = [ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
 tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))]

Freeze the base model layers

In [ ]:
for base_model in base_models:
  print(f"Training with {base_model} pretrained layers")
  for layer in base_model.layers:
      layer.trainable = False

In [ ]:
  # Create your custom classification head
  num_classes = len(set(y_train))

In [ ]:
  model = models.Sequential([
      base_model,
      layers.GlobalAveragePooling2D(),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(num_classes, activation='softmax')
  ])

In [ ]:
  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
  # Train the model
  epochs = 10  # Adjust the number of epochs as needed
  model.fit(
      train_generator,
      epochs=epochs,
      validation_data=validation_generator
  )

In [ ]:
  # Evaluate the model
  test_loss, test_acc = model.evaluate(test_generator)
  print(f'Test accuracy: {test_acc}')

In [ ]:
  if test_acc < 1:
    # Make predictions on the test set
    predictions = model.predict(test_generator)

    # Get the predicted labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Get the true labels
    true_labels = test_generator.classes

    # Find indices of incorrect and correct predictions
    incorrect_indices = np.where(predicted_labels != true_labels)[0]
    correct_indices = np.where(predicted_labels == true_labels)[0]

    # Extract corresponding image paths for reporting
    incorrect_image_paths = df_test.iloc[incorrect_indices]['image_path'].tolist()
    correct_image_paths = df_test.iloc[correct_indices]['image_path'].tolist()

    # Report incorrect and correct classified examples
    print("Incorrectly Classified Examples:")
    for image_path in incorrect_image_paths:
        print(image_path)
    print("\nCorrectly Classified Examples:")
    for image_path in correct_image_paths:
        print(image_path)


##Fine Tuning:<br>
###First Pre-trained Layer Model - ResNet50<br>
1. Training accuracy reaches 67.22% after 10 epochs.<br>
2. Test accuracy is 68.85%, which doesn't seems like great accuracy given we are using pre-trained model.<br>
3. We oberved val accuracy of 52.46% which implies the ResNet with frozen layers is underfitting, so we tried making the model a little more complex with increasing the number of neurons in additional dense layers from 128 to 256 and 512, but the accuracy still didn't go up significantly.<br>
4. We tried early stopping and reducing LR on plateau techniques along with Adam optimizer and dropout layer for fine tuning and optimization but still the accuracy went upto max 82% with a random split of train, validation and test set case.<br>
5. Then, based on results we tried to use different set of pretrained layer model i.e. Google's InceptionV3<br>
###Second Pre-trained Layer Model - InceptionV3<br>
1. The model achieves strong performance on both the training and validation sets.<br>
2. The test accuracy is also high at 95.08%, suggesting good generalization to unseen data.<br>
3. The training process seems stable with consistent improvement in both loss and accuracy over epochs.<br>
4. For both the models we used preprocess_input from vgg16 pretrained model, the preprocess input from vgg16 combined with InceptionV3 model gave good results (95% accuracy), with frozen layers and GlobalAveragePoolingLayer, Dense layer on top with 128 neurons and 0.5 dropout along with Adam optimizer while compiltation.<br>
